In [16]:
import xarray as xr
import os
import netCDF4
import numpy as np
import torch
from torch import nn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torch.utils.data import random_split
import random
import pandas as pd
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau

# combine mli/mlo data to one nc file

In [2]:
# from os import listdir
# from os.path import isfile, join

# mypath = '/work/sds-lab/Shuochen/climsim/train'
# # allfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
# allfiles = []
# for path, subdirs, files in os.walk(mypath):
#     for name in files:
#        allfiles.append(os.path.join(path, name))
# print(len(allfiles))

# mli_files = []
# mlo_files = []

# for i in range(len(allfiles)):
#     if 'mli' in allfiles[i]:
#         mli_files.append(allfiles[i])
#     if 'mlo' in allfiles[i]:
#         mlo_files.append(allfiles[i])

# ds_mli = []
# ds_mlo = []

# for i in range(len(mli_files)):
#     ds_mli.append(xr.open_dataset(mli_files[i]).expand_dims('time'))
# for i in range(len(mlo_files)):
#     ds_mlo.append(xr.open_dataset(mlo_files[i]).expand_dims('time'))

# combined_mli = xr.concat(ds_mli, dim='time').to_netcdf(/work/sds-lab/Shuochen/climsim/ + 'val_mli.nc')
# combined_mlo = xr.concat(ds_mlo, dim='time').to_netcdf('/work/sds-lab/Shuochen/climsim/' + 'val_mlo.nc')

# preprocessing

In [17]:
mli = xr.open_dataset('/work/sds-lab/Shuochen/climsim/val_mli.nc')
mlo = xr.open_dataset('/work/sds-lab/Shuochen/climsim/val_mlo.nc')

In [18]:
# fix time index
date = []
for i in range(len(mli.ymd.values)):
    my_str=str(mli.ymd.values[i])
    date.append(my_str[:0] + '200' + my_str[0:])

mli['time'] = pd.to_datetime(date) + pd.to_timedelta(mli.tod.values, unit='s')
mlo['time'] = pd.to_datetime(date) + pd.to_timedelta(mlo.tod.values, unit='s')

In [19]:
# downsample data every 7 samples (select 1 every 7 samples), data frequency is 1200s.
mli = mli.isel(time=slice(None,None,7))
mlo = mlo.isel(time=slice(None,None,7))

In [23]:
# in/out variable lists (6:10)
vars_mli = ['state_t','state_q0001','state_ps','pbuf_SOLIN', 'pbuf_LHFLX', 'pbuf_SHFLX']
vars_mlo = ['ptend_t','ptend_q0001','cam_out_NETSW','cam_out_FLWDS','cam_out_PRECSC','cam_out_PRECC','cam_out_SOLS','cam_out_SOLL','cam_out_SOLSD','cam_out_SOLLD']

# nc to numpy to tensor, then reshape
input_list = []
output_list = []

for i in range(len(vars_mli)):
    if i < 2:
        temp=torch.Tensor(mli[vars_mli[i]].to_numpy())
        new_var=temp.permute(0,2,1)
        new_var=new_var.reshape(new_var.shape[0] * new_var.shape[1], new_var.shape[2])
        input_list.append(new_var)
    else:
        temp=torch.Tensor(mli[vars_mli[i]].to_numpy())
        new_var=temp.reshape(temp.shape[0] * temp.shape[1], 1)
        input_list.append(new_var)

# calculate heating and moistening tendency.
ptend_t = (mlo['state_t'] - mli['state_t']) / 1200
ptend_q0001 = (mlo['state_q0001'] - mli['state_q0001']) / 1200

ptend_t = torch.Tensor(ptend_t.to_numpy()).permute(0,2,1)
ptend_t = ptend_t.reshape(ptend_t.shape[0] * ptend_t.shape[1], ptend_t.shape[2])

ptend_q0001 = torch.Tensor(ptend_q0001.to_numpy()).permute(0,2,1)
ptend_q0001 = ptend_q0001.reshape(ptend_q0001.shape[0] * ptend_q0001.shape[1], ptend_q0001.shape[2])

# add to output list
output_list.extend([ptend_t,ptend_q0001])

for i in range(len(vars_mlo)):
    if i >= 2:
        temp=torch.Tensor(mlo[vars_mlo[i]].to_numpy())
        new_var=temp.reshape(temp.shape[0] * temp.shape[1], 1)
        output_list.append(new_var)

In [28]:
# normalization and scaling
norm_mean = xr.open_dataset('/work/sds-lab/Shuochen/climsim/normalizations_git/inputs/input_mean.nc')
norm_max = xr.open_dataset('/work/sds-lab/Shuochen/climsim/normalizations_git/inputs/input_max.nc')
norm_min = xr.open_dataset('/work/sds-lab/Shuochen/climsim/normalizations_git/inputs/input_min.nc')
scale = xr.open_dataset('/work/sds-lab/Shuochen/climsim/normalizations_git/outputs/output_scale.nc')

norm_mean_list = []
for i in range(len(vars_mli)):
    temp=torch.Tensor(norm_mean[vars_mli[i]].values)
    norm_mean_list.append(temp)

norm_max_list = []
for i in range(len(vars_mli)):
    temp=torch.Tensor(norm_max[vars_mli[i]].values)
    norm_max_list.append(temp)

norm_min_list = []
for i in range(len(vars_mli)):
    temp=torch.Tensor(norm_min[vars_mli[i]].values)
    norm_min_list.append(temp)

scale_list = []
for i in range(len(vars_mlo)):
    temp=torch.Tensor(scale[vars_mlo[i]].values)
    scale_list.append(temp)

In [31]:
for i in range(len(input_list)):
    input_list[i] = (input_list[i] - norm_mean_list[i]) / (norm_max_list[i] - norm_min_list[i])  

for i in range(len(output_list)):
    output_list[i] = output_list[i] * scale_list[i]

In [33]:
# concat
input = torch.cat((input_list[0],input_list[1],input_list[2],input_list[3],input_list[4],input_list[5]), dim=1)
output = torch.cat((output_list[0],output_list[1],output_list[2],output_list[3],output_list[4],output_list[5],output_list[6],output_list[7],output_list[8],output_list[9]), dim=1)

print(input.shape,output.shape)

torch.Size([1441920, 124]) torch.Size([1441920, 128])


# test input/output data with theirs (val:0008-02 to 0009-01)

In [34]:
X=np.load('/work/sds-lab/Shuochen/climsim/val_input.npy')
y=np.load('/work/sds-lab/Shuochen/climsim/val_target.npy')

In [35]:
X.shape

(1441920, 124)

In [36]:
X[0,:]

array([-2.1913459e-03, -1.7944887e-02, -4.5234229e-02, -1.8812425e-02,
        1.2241836e-03,  5.4736424e-02,  7.9991966e-02,  6.6402398e-02,
        6.1990730e-02,  6.2253091e-02,  5.1185194e-02,  4.6816483e-02,
        4.8013736e-02,  4.2244118e-02,  1.9963728e-02, -1.1743228e-02,
       -1.6514115e-02, -3.1537268e-02, -7.3459931e-02, -8.7187998e-02,
       -9.5602296e-02, -6.7409396e-02, -2.5630416e-02,  1.4921249e-02,
        4.1257340e-02,  5.6141108e-02,  6.2658504e-02,  6.4611822e-02,
        6.6882201e-02,  6.7682333e-02,  6.9309019e-02,  7.5467244e-02,
        8.1024468e-02,  8.4922343e-02,  8.7200657e-02,  8.7108433e-02,
        8.9283474e-02,  9.1625057e-02,  9.4715320e-02,  9.5724754e-02,
        9.7043157e-02,  9.8063178e-02,  9.7116463e-02,  9.8507255e-02,
        9.8593235e-02,  9.7905174e-02,  9.5312789e-02,  9.1622062e-02,
        8.7270044e-02,  8.4584579e-02,  8.3577700e-02,  8.3638921e-02,
        8.3342053e-02,  8.2297072e-02,  8.0700077e-02,  7.8771718e-02,
      

In [37]:
input[0,:]

tensor([-2.1913e-03, -1.7945e-02, -4.5234e-02, -1.8812e-02,  1.2242e-03,
         5.4736e-02,  7.9992e-02,  6.6402e-02,  6.1991e-02,  6.2253e-02,
         5.1185e-02,  4.6817e-02,  4.8014e-02,  4.2244e-02,  1.9964e-02,
        -1.1743e-02, -1.6514e-02, -3.1537e-02, -7.3460e-02, -8.7188e-02,
        -9.5602e-02, -6.7410e-02, -2.5630e-02,  1.4921e-02,  4.1257e-02,
         5.6141e-02,  6.2659e-02,  6.4612e-02,  6.6882e-02,  6.7682e-02,
         6.9309e-02,  7.5467e-02,  8.1025e-02,  8.4922e-02,  8.7201e-02,
         8.7108e-02,  8.9283e-02,  9.1625e-02,  9.4715e-02,  9.5725e-02,
         9.7043e-02,  9.8063e-02,  9.7116e-02,  9.8507e-02,  9.8593e-02,
         9.7905e-02,  9.5313e-02,  9.1622e-02,  8.7270e-02,  8.4585e-02,
         8.3578e-02,  8.3639e-02,  8.3342e-02,  8.2297e-02,  8.0700e-02,
         7.8772e-02,  7.5982e-02,  7.3353e-02,  6.9023e-02,  6.4707e-02,
         3.3672e-01,  3.2875e-01,  3.1961e-01,  3.0869e-01,  2.9163e-01,
         2.7643e-01,  2.5446e-01,  2.2798e-01,  1.8

In [38]:
y[0,:]

array([-1.36620812e-02, -2.80409772e-02, -2.15656627e-02, -5.23943938e-02,
       -8.13384652e-02, -1.10813349e-01, -1.06115066e-01, -7.64378756e-02,
       -4.98877764e-02, -3.91743705e-02, -2.72510257e-02, -1.97552871e-02,
       -1.45448223e-02, -1.03805251e-02, -5.29604033e-03, -7.86766876e-04,
        1.00812223e-03,  1.03712187e-03,  2.60534324e-03,  6.22053689e-04,
       -9.80621320e-04, -4.21181973e-03, -8.36136844e-03, -1.23731298e-02,
       -1.74749307e-02, -2.14770287e-02, -2.19386593e-02, -2.27263030e-02,
       -2.29678582e-02, -2.25563366e-02, -2.36029029e-02, -2.46444885e-02,
       -2.34153941e-02, -2.36174222e-02, -2.38939375e-02, -2.45916806e-02,
       -2.31757555e-02, -2.32806299e-02, -2.33938787e-02, -2.34074965e-02,
       -2.29851063e-02, -2.21269783e-02, -2.35317610e-02, -2.45282371e-02,
       -2.89273020e-02, -2.49604452e-02, -2.53472887e-02, -2.64138561e-02,
       -2.98754536e-02, -2.96021532e-02, -3.14517878e-02, -3.01577393e-02,
       -2.94105019e-02, -

In [39]:
output[0,:]

tensor([-1.3662e-02, -2.8041e-02, -2.1566e-02, -5.2394e-02, -8.1338e-02,
        -1.1081e-01, -1.0612e-01, -7.6438e-02, -4.9888e-02, -3.9174e-02,
        -2.7251e-02, -1.9755e-02, -1.4545e-02, -1.0381e-02, -5.2960e-03,
        -7.8677e-04,  1.0081e-03,  1.0371e-03,  2.6053e-03,  6.2205e-04,
        -9.8062e-04, -4.2118e-03, -8.3614e-03, -1.2373e-02, -1.7475e-02,
        -2.1477e-02, -2.1939e-02, -2.2726e-02, -2.2968e-02, -2.2556e-02,
        -2.3603e-02, -2.4644e-02, -2.3415e-02, -2.3617e-02, -2.3894e-02,
        -2.4592e-02, -2.3176e-02, -2.3281e-02, -2.3394e-02, -2.3407e-02,
        -2.2985e-02, -2.2127e-02, -2.3532e-02, -2.4528e-02, -2.8927e-02,
        -2.4960e-02, -2.5347e-02, -2.6414e-02, -2.9875e-02, -2.9602e-02,
        -3.1452e-02, -3.0158e-02, -2.9411e-02, -2.9973e-02, -3.1699e-02,
        -3.2874e-02, -3.2675e-02, -3.4301e-02, -3.8884e-02, -4.0407e-02,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0